In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as w
from pytransform3d.plot_utils import make_3d_axis
from pytransform3d.transformations import plot_transform, transform_from
from pytransform3d.rotations import matrix_from_euler

# --- Figure & Axes ---
ax = make_3d_axis(ax_s=1.0)
ax.set_title("Interactive Pose: xyz + roll/pitch/yaw")
ax.set_xlim(-1,1); ax.set_ylim(-1,1); ax.set_zlim(-1,1)
ax.set_box_aspect([1,1,1])

In [ ]:
# --- Widgets ---
sx=w.FloatSlider(min=-1,max=1,step=0.01,description='x',value=0.0)
sy=w.FloatSlider(min=-1,max=1,step=0.01,description='y',value=0.0)
sz=w.FloatSlider(min=-1,max=1,step=0.01,description='z',value=0.0)
sroll=w.FloatSlider(min=-180,max=180,step=1,description='roll°',value=0.0)
spitch=w.FloatSlider(min=-180,max=180,step=1,description='pitch°',value=0.0)
syaw=w.FloatSlider(min=-180,max=180,step=1,description='yaw°',value=0.0)
L=w.FloatSlider(min=0.05,max=0.5,step=0.01,value=0.2,description='axis_len')
show_world=w.Checkbox(value=True,description='显示世界坐标轴')
reset_btn=w.Button(description='重置')

def draw_pose(*_):
    ax.cla()
    ax.set_title("Interactive Pose: xyz + roll/pitch/yaw")
    ax.set_xlim(-1,1); ax.set_ylim(-1,1); ax.set_zlim(-1,1)
    ax.set_box_aspect([1,1,1])

    if show_world.value:
        plot_transform(ax, A2B=np.eye(4), s=0.25)

    r,p,y = np.deg2rad([sroll.value, spitch.value, syaw.value])
    R = matrix_from_euler([r,p,y],0,1,2,True)
    t = np.array([sx.value, sy.value, sz.value])
    T = transform_from(R, t)
    plot_transform(ax, A2B=T, s=L.value)
    ax.figure.canvas.draw_idle()

def on_reset(_):
    sx.value = sy.value = sz.value = 0.0
    sroll.value = spitch.value = syaw.value = 0.0
    L.value = 0.2
    show_world.value = True

In [ ]:
for wid in (sx,sy,sz,sroll,spitch,syaw,L,show_world):
    wid.observe(draw_pose, names='value')
reset_btn.on_click(on_reset)

controls = w.VBox([
    w.HBox([sx, sy, sz]),
    w.HBox([sroll, spitch, syaw]),
    w.HBox([L, show_world, reset_btn]),
])

# 关键：把“图像画布”当成一个 widget 和控件一起显示
app = w.VBox([controls, ax.figure.canvas])   # 想左右排就换成 w.HBox([...])
display(app)

draw_pose()